In [1]:
#install dependences
#!pip install dateutil
#!pip install pandas
#!pip install yfinance
#!pip install pathlib
from datetime import datetime as dt
from datetime import timedelta
import os
import os.path
import pandas as pd
from pathlib import Path
import dateutil.parser as dparser

In [2]:
#data scrubbing and separation
#check if production file exists, run if not
if not os.path.exists("OilSpills-Production.csv"):   

    #open master file, separate rows with values in Trading Name from those without
    #and drop excluded data to csv as it is done with
    
    df_master = pd.read_csv("OilSpills-Master.csv")
    
    df = df_master[df_master["Trading Name"].notna()]
    df_excluded = df_master[df_master["Trading Name"].isna()]    
    
    df_excluded.to_csv(r"OilSpills-Excluded.csv", index = False, header=True)
   
    #date correction to production dataframe      
    thisYear = int(dt.today().strftime("%Y"))
         
    for index, row in df.iterrows():         
        
        if " " not in row.Dates:
            
            rawSingle = (dparser.parse(row.Dates))  
            fixedSingle = rawSingle.strftime('%Y-%m-%d')
            fixedDate = fixedSingle + " - " + fixedSingle        
        
        else:
            
            rawDoubleSplit = row.Dates.split(" - ")
            rawSplitFirst = (dparser.parse(rawDoubleSplit[0]))
            rawSplitSecond = (dparser.parse(rawDoubleSplit[1]))
            fixedSplitFirst = rawSplitFirst.strftime('%Y-%m-%d')
            fixedSplitSecond = rawSplitSecond.strftime('%Y-%m-%d')
            fixedDate = fixedSplitFirst + " - " + fixedSplitSecond  
    
        #Fix dates if prior to 1970
        splitFixed = fixedDate.split(" - ")
        
        splitFirstDate = splitFixed[0].split("-")
        splitFirstYear = int(splitFirstDate[0])
        
        splitSecondDate = splitFixed[1].split("-")
        splitSecondYear = int(splitSecondDate[0])
        
        if splitFirstYear > thisYear or splitSecondYear > thisYear:
            
            if splitFirstYear > thisYear:
                splitFirstDate[0] = splitFirstYear - 100
                firstDate = str(splitFirstDate[0]) + "-" + splitFirstDate[1] + "-" + str(splitFirstDate[2])                

            if splitSecondYear > thisYear:
                splitSecondDate[0] = splitSecondYear - 100
                secondDate = str(splitSecondDate[0]) + "-" + splitSecondDate[1] + "-" + str(splitSecondDate[2])
                
            fixedDate = firstDate + " - " + secondDate
                

        df.at[index, "Dates"] = fixedDate
            
    df.to_csv(r"OilSpills-Production.csv", index = False, header=True)   

In [ ]:
#stock data
import yfinance as yf
import dateutil.parser as dparser
import os.path

df = pd.read_csv("OilSpills-Production.csv")

#check if stocks subfolder exists, otherwise create it (for a bit more organization)   
if not os.path.exists("stocks"):
        os.makedirs("stocks")

#get date ranges for each incident (2 weeks before and 2 weeks after) for each incident
#then pull stock data for those data ranges and save to csv

for index, row in df.iterrows():
    
    tradingName = row["Trading Name"]
    dates = row["Dates"]
    dateSplit = dates.split(" - ")     
    dateStartDelta = (dparser.parse(dateSplit[0])) - timedelta(days=14)
    dateEndDelta = (dparser.parse(dateSplit[1])) + timedelta(days=14)
    dateStart = dateStartDelta.strftime('%Y-%m-%d')
    dateEnd = dateEndDelta.strftime('%Y-%m-%d') 
    
    stockTarget = (f"{tradingName} - {dateStart} - {dateEnd}.csv")
    
#check if the csvs we have match up with the needs of the production sheet
#should support both unix and Windows OS file structures
    pathTarget = Path(f"stocks/{stockTarget}")
    print(pathTarget)
    if not os.path.exists(pathTarget):
        try:
            data = yf.download(tradingName, start=dateStart, end=dateEnd)
            data.to_csv(fr"{pathTarget}", index = False, header=True)
            
        except Exception as exception:
            print(f"{stockTarget} encountered an error.  Please review the error message and adjust \
                  master sheet accordingly.  Then remove production sheet and re-run the notebook.")
            print("\n")
            print(f"print {exception}")
            pass


stocks\NILSY - 2020-05-15 - 2020-06-12.csv
[*********************100%***********************]  1 of 1 completed
stocks\GRKA - 2020-03-16 - 2020-04-13.csv
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- GRKA: No data found for this date range, symbol may be delisted
stocks\TRP - 2019-10-15 - 2019-11-12.csv
[*********************100%***********************]  1 of 1 completed
stocks\CAP.SN - 2019-07-13 - 2019-08-10.csv
[*********************100%***********************]  1 of 1 completed
stocks\HSE.TO - 2018-11-02 - 2018-11-30.csv
[*********************100%***********************]  1 of 1 completed
stocks\TRP - 2017-11-02 - 2017-11-30.csv
[*********************100%***********************]  1 of 1 completed
stocks\ED - 2017-04-23 - 2017-05-21.csv
[*********************100%***********************]  1 of 1 completed
stocks\SEMG - 2017-03-21 - 2017-04-18.csv
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- SEM

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- SUN: Data doesn't exist for startDate = 281682000, endDate = 284101200
stocks\COP - 1977-04-08 - 1977-05-06.csv
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- COP: Data doesn't exist for startDate = 229320000, endDate = 231739200
stocks\RDS-A - 1974-07-26 - 1974-08-23.csv
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- RDS-A: Data doesn't exist for startDate = 144043200, endDate = 146462400
stocks\CVX - 1970-01-27 - 1970-02-24.csv
[*********************100%***********************]  1 of 1 completed

Exception in thread Thread-141:
Traceback (most recent call last):
  File "C:\Users\trbie\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\trbie\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\trbie\anaconda3\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Users\trbie\anaconda3\lib\site-packages\yfinance\multi.py", line 166, in _download_one_threaded
    data = _download_one(ticker, start, end, auto_adjust, back_adjust,
  File "C:\Users\trbie\anaconda3\lib\site-packages\yfinance\multi.py", line 178, in _download_one
    return Ticker(ticker).history(period=period, interval=interval,
  File "C:\Users\trbie\anaconda3\lib\site-packages\yfinance\base.py", line 121, in history
    start = int(_time.mktime(
OverflowError: mktime argument out of range



stocks\BP - 1967-03-04 - 1967-04-01.csv
